# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load drawingTools.py

In [3]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [4]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v101'
version_v8 = 'v112'

version_v9 = 'v113'

# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v75t', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v75t', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

# samples_nugus = [
#     Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
#     Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
#     ]


samples_nugunrates = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version, 'new-clusters')
    ]


samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_v8, 'old-ID')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_v9, 'new-ID')
    ]








gen_part_selections: 8


In [5]:
do_rate = True

In [6]:
# %load settings.py
# === samples =====================================================
import python.selections as selections
import python.collections as collections

import pprint


samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_nugunrates_V9

for smp in samples:
    smp.open_file()


sample = 'new-ID'

do_rate = True

# === TP ==========================================================
# tps = ['HMvDR', 'EG', 'TkEG', 'TkEle', 'TkIsoEle']
# tps = ['HMvDR', 
#        'EG', 
#        'TkEG', 
#        'TkEle', 
#        'TkIsoEle',
#        'HMvDR', 
#        'HMvDRCalib',
#        'HMvDRcylind5Calib',
#        'HMvDRcylind2p5Calib',
#        'HMvDRshapeCalib',
#        'HMvDRshapeDrCalib',
#        'HMvDRCalibMerged',
#        'HMvDRshapeCalibMerged']

tps = [
#     'HMvDR', 
#        'HMvDRCalib',
#        'HMvDRcylind5Calib',
#        'HMvDRcylind2p5Calib',
#        'HMvDRshapeCalib',
#        'HMvDRshapeDrCalib',
#        'HMvDREmInt',
#        'HMvDREmIntMerged',
       'EG',
       'TkEle',
       'TkEleEL',
    
#        'HMvDRCalibMerged',
#        'HMvDRshapeCalibMerged'
      ]

# === Load the Histo Primitives ====================================
histo_primitives = pd.DataFrame() 

if do_rate:
    from python.plotters import rate_plotters, eg_rate_plotters

    for plotter in rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in eg_rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.data == tp].data_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.data == tp].gen_sel.unique().tolist()

#  ==== labels ===============================================
tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]



import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


# print selections.eg_rate_selections

[EventManager] registering collection: MC
[EventManager] registering collection: GEN
[EventManager] registering collection: TC
[EventManager] registering collection: TCTrue
[EventManager] registering collection: DEF2D
[EventManager] registering collection: DEF2DTrue
[EventManager] registering collection: HMvDRTrue
[EventManager] registering collection: DEF
[EventManager] registering collection: DEFNC
[EventManager] registering collection: HMvDR
[EventManager] registering collection: HMvDREmInt
[EventManager] registering collection: HMvDREmIntMerged
[EventManager] registering collection: HMvDRRebin
[EventManager] registering collection: HMvDRsTC
[EventManager] registering collection: HMvDRNC0
[EventManager] registering collection: HMvDRNC1
[EventManager] registering collection: DEFMerged
[EventManager] registering collection: DEFCalib
[EventManager] registering collection: HMvDRMerged
[EventManager] registering collection: HMvDRfixed
[EventManager] registering collection: HMvDRcylind10


In [7]:
print histo_primitives.data.unique()


['HMvDR' 'HMvDREmInt' 'HMvDREmIntMerged' 'EG' 'TkEle' 'TkEleEL' 'TkEleBRL'
 'TkEleELBRL']


In [8]:
histo_primitives[:10]

,data,data_sel,gen_sel,data_label,data_sel_label,gen_sel_label
0,HMvDR,all,nomatch,HM+dR(layer) Cl3d,,
1,HMvDR,EtaD,nomatch,HM+dR(layer) Cl3d,2.4 < |#eta^{L1}| <= 2.8,
2,HMvDR,EtaBC,nomatch,HM+dR(layer) Cl3d,1.52 < |#eta^{L1}| <= 2.4,
3,HMvDR,EtaBCD,nomatch,HM+dR(layer) Cl3d,1.52 < |#eta^{L1}| <= 2.8,
4,HMvDR,Em,nomatch,HM+dR(layer) Cl3d,EGId,
5,HMvDR,EmEtaD,nomatch,HM+dR(layer) Cl3d,"EGId, 2.4 < |#eta^{L1}| <= 2.8",
6,HMvDR,EmEtaBC,nomatch,HM+dR(layer) Cl3d,"EGId, 1.52 < |#eta^{L1}| <= 2.4",
7,HMvDR,EmEtaBCD,nomatch,HM+dR(layer) Cl3d,"EGId, 1.52 < |#eta^{L1}| <= 2.8",
8,HMvDREmInt,all,nomatch,HM+dR(layer) Cl3d EM Int,,
9,HMvDREmInt,EtaD,nomatch,HM+dR(layer) Cl3d EM Int,2.4 < |#eta^{L1}| <= 2.8,


In [9]:
tp_labels

{'EG': 'EG',
 'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDREmInt': 'HM+dR(layer) Cl3d EM Int',
 'HMvDREmIntMerged': 'HM+dR(layer) Cl3d EM Int Merged',
 'TkEle': 'TkEle',
 'TkEleBRL': 'TkEle B',
 'TkEleEL': 'TkEle ELLIPTIC',
 'TkEleELBRL': 'TkEle ELLIPTIC B'}

In [10]:






tp_selection_labels


{'EGq2': 'hwQual 2',
 'EGq2EtaBC': 'hwQual 2, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq2EtaBCD': 'hwQual 2, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq2EtaD': 'hwQual 2, 2.4 < |#eta^{L1}| <= 2.8',
 'EGq2Iso0p1': 'hwQual 2, Iso0p1',
 'EGq2Iso0p1EtaBC': 'hwQual 2, Iso0p1, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq2Iso0p1EtaBCD': 'hwQual 2, Iso0p1, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq2Iso0p1EtaD': 'hwQual 2, Iso0p1, 2.4 < |#eta^{L1}| <= 2.8',
 'EGq2Iso0p2': 'hwQual 2, Iso0p2',
 'EGq2Iso0p2EtaBC': 'hwQual 2, Iso0p2, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq2Iso0p2EtaBCD': 'hwQual 2, Iso0p2, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq2Iso0p2EtaD': 'hwQual 2, Iso0p2, 2.4 < |#eta^{L1}| <= 2.8',
 'EGq3': 'hwQual 3',
 'EGq3EtaBC': 'hwQual 3, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq3EtaBCD': 'hwQual 3, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq3EtaD': 'hwQual 3, 2.4 < |#eta^{L1}| <= 2.8',
 'EGq3Iso0p1': 'hwQual 3, Iso0p1',
 'EGq3Iso0p1EtaBC': 'hwQual 3, Iso0p1, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq3Iso0p1EtaBCD': 'hwQual 3, Iso0p1, 1.52 < |#eta^{L1}| <= 2.8',
 '

In [11]:
gen_selection_labels

{'nomatch': ''}

In [12]:
%%time
hplot = HPlot(samples, tp_labels, tp_selection_labels, [])

for tp in tps:
    print tp
    print histo_primitives[histo_primitives.data == tp].data_sel.unique()
    print samples
    hplot.cache_histo(classtype=histos.RateHistos, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=None)


EG
['EGq2' 'EGq2EtaD' 'EGq2EtaBC' 'EGq2EtaBCD' 'EGq3' 'EGq3EtaD' 'EGq3EtaBC'
 'EGq3EtaBCD' 'EGq4' 'EGq4EtaD' 'EGq4EtaBC' 'EGq4EtaBCD' 'EGq5' 'EGq5EtaD'
 'EGq5EtaBC' 'EGq5EtaBCD']
[<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID>]
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID>
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq2 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq2EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq2EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq2EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq3 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq3EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq3EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> EG EGq3EtaBCD None
<Samp

<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq2Iso0p1EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3Iso0p2 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3Iso0p2EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3Iso0p2EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3Iso0p2EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> TkEleEL EGq3Iso0p1 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v113.root, new-ID> 

In [13]:
hplot.data['sample'].unique()

array(['new-ID'], dtype=object)

In [14]:
# sample='new-ID'

In [15]:

for tp in tps:
    for tp_sel in tp_select[tp]:
        print tp,tp_sel
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)


EG EGq2
normalize to 31000
EG EGq2EtaD
normalize to 31000
EG EGq2EtaBC
normalize to 31000
EG EGq2EtaBCD
normalize to 31000
EG EGq3
normalize to 31000
EG EGq3EtaD
normalize to 31000
EG EGq3EtaBC
normalize to 31000
EG EGq3EtaBCD
normalize to 31000
EG EGq4
normalize to 31000
EG EGq4EtaD
normalize to 31000
EG EGq4EtaBC
normalize to 31000
EG EGq4EtaBCD
normalize to 31000
EG EGq5
normalize to 31000
EG EGq5EtaD
normalize to 31000
EG EGq5EtaBC
normalize to 31000
EG EGq5EtaBCD
normalize to 31000
TkEle EGq2
normalize to 31000
TkEle EGq2EtaD
normalize to 31000
TkEle EGq2EtaBC
normalize to 31000
TkEle EGq2EtaBCD
normalize to 31000
TkEle EGq2Iso0p2
normalize to 31000
TkEle EGq2Iso0p2EtaD
normalize to 31000
TkEle EGq2Iso0p2EtaBC
normalize to 31000
TkEle EGq2Iso0p2EtaBCD
normalize to 31000
TkEle EGq2Iso0p1
normalize to 31000
TkEle EGq2Iso0p1EtaD
normalize to 31000
TkEle EGq2Iso0p1EtaBC
normalize to 31000
TkEle EGq2Iso0p1EtaBCD
normalize to 31000
TkEle EGq3
normalize to 31000
TkEle EGq3EtaD
normalize 

#### 3D clusters matched to GEN

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_norm for his in hsets], labels, text=text)
         

In [16]:
version_V9 = 'v113'
scaling_params =  pd.read_csv(r'scaling_params_{}.csv'.format(version_V9))

In [17]:
scaling_params

,Unnamed: 0,tp,tp_sel,params,pu
0,0,EG,EGq3EtaBC,"(4.389777288157897, 1.076240920512368)",PU0
1,1,EG,EGq4EtaBC,"(9.691849364408037, 1.1194287172322557)",PU0
2,2,EG,EGq5EtaBC,"(5.844221143785292, 1.1075267334974994)",PU0
3,3,TkEleEL,EGq3EtaBC,"(3.555303964339816, 1.101932349217412)",PU0
4,4,TkEleEL,EGq4EtaBC,"(8.250063797793905, 1.162328265666231)",PU0
5,5,TkEleEL,EGq5EtaBC,"(3.4292877337784082, 1.204655775508082)",PU0
6,6,EG,EGq3EtaBC,"(7.622337833437538, 0.9458618852654797)",PU200
7,7,EG,EGq4EtaBC,"(10.62852445676714, 1.061621871489299)",PU200
8,8,EG,EGq5EtaBC,"(8.72916915396328, 1.0080172914768089)",PU200
9,9,TkEleEL,EGq3EtaBC,"(4.0158888558935555, 1.062947253698448)",PU200


In [18]:
eval(scaling_params[(scaling_params.pu == 'PU200') & (scaling_params.tp == 'EG') & (scaling_params.tp_sel == 'EGq3EtaBC')].params.iloc[0])

(7.622337833437538, 0.9458618852654797)

In [21]:
def convertRateToGraph(histo_rate, params=None):
    xaxis = histo_rate.GetXaxis()
    fNpoints = xaxis.GetNbins()
    fX = [0]*fNpoints
    fEX = [0]*fNpoints
    fY = [0]*fNpoints
    fEY = [0]*fNpoints

    graph = ROOT.TGraphAsymmErrors()
    graph.Set(fNpoints)
    for i in range(0, fNpoints):
        if params is None:
            fX[i] = xaxis.GetBinLowEdge(i + 1)
        else:
            fX[i] = params[0] + params[1]*xaxis.GetBinLowEdge(i + 1)
        fY[i] = histo_rate.GetBinContent(i + 1)
        fEX[i] = histo_rate.GetBinWidth(i + 1) * ROOT.gStyle.GetErrorX()
        fEY[i] = histo_rate.GetBinError(i + 1)
    return fX, fY, fEX, fEX, fEY, fEY


In [23]:
for tp in ['EG', 'TkEleEL']:
    for tp_sel in ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC']:

        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)

        draw([his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True)
        draw([his.h_pt_graph('offline', 'rate [kHz]', lambda x: convertRateToGraph(x, eval(scaling_params[(scaling_params.pu == 'PU200') & 
                                                                                                  (scaling_params.tp == tp) & 
                                                                                                  (scaling_params.tp_sel == tp_sel)].params.iloc[0]))) for his in hsets], 
             labels, text=text,  min_y =1, max_y = 40000 ,logy=True)








In [27]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'EG', ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='offline p_{T} thresh. [GeV]')

<ROOT.TCanvas object ("c_e925d7") at 0x7ff4ad6547c0>

In [29]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'TkEleEL', ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='offline p_{T} thresh. [GeV]')

<ROOT.TCanvas object ("c_917fbe") at 0x7ff4ad1a1960>

In [ ]:
for tp in ['EG', 'TkEleEL']:
    for tp_sel in ['EGq3EtaBC', 'EGq5EtaBC']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_pt for his in hsets], labels, text=text, logy=True)
         

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDR', 'HMvDREmInt' ], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDR', 'HMvDRCalib', 'HMvDREmInt' ], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
sample

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG', 'TkEle', 'TkEleEL'], ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG'], ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
sample

In [ ]:
#HERE
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID', 'old-ID', 'new-clusters'], 'PU200', ['EG'], ['EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
labels = ['TP head', 'L1 menu version', 'backported']
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID'], 'PU200', ['EG'], ['EGq3EtaBC', 'EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID'], 'PU200', ['EG', 'TkEle','TkEleEL'], ['EGq3EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID'], 'PU200', ['TkEleEL'], ['EGq3EtaBC', 'EGq3Iso0p2EtaBC', 'EGq3Iso0p3EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID', 'old-ID'], 'PU200', ['EG'], ['EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEle'], ['EGq3EtaBC', 'EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
# 'HMvDRCalib','HMvDRshapeDrCalib', 
final_haset = []
labels_final = []
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EtaBCD', None)
for hist in hsets:
    hist.normalize(31000)
final_haset.append(hsets[0]) 
labels_final.append('all clusters')
# hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EmEtaBCD', None)
# for hist in hsets:
#     hist.normalize(31000)
# final_haset.append(hsets[0]) 
# labels_final.append('EG ID clusters')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDREmInt'], 'EmEtaBCD', None)
for hist in hsets:
    hist.normalize(31000)
final_haset.append(hsets[0]) 
labels_final.append('passing EG ID')
# print text
draw([his.h_pt for his in final_haset], labels_final, text='PU200, 1.52 < |#eta^{L1}| <= 2.8', logy=True,
     do_write=True, write_name='egammEndcap_EGID_EtaBCD_rate')

In [ ]:
final_haset = []
labels_final = []
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EtaBC', None)
for hist in hsets:
    hist.normalize(31000)
final_haset.append(hsets[0]) 
labels_final.append('all clusters')
# hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EmEtaBC', None)
# for hist in hsets:
#     hist.normalize(31000)
# final_haset.append(hsets[0]) 
# labels_final.append('EG ID clusters')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDREmInt'], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
final_haset.append(hsets[0]) 
labels_final.append('passing EG ID')
# print text
draw([his.h_pt for his in final_haset], labels_final, text='PU200, 1.52 < |#eta^{L1}| <= 2.4', logy=True,
     do_write=True, write_name='egammEndcap_EGID_EtaBC_rate')

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEle', 'EG'], 'EGq4EtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels=['EC standalone', 'EC + L1 Tk'], text=text, logy=True,
     do_write=True, write_name='egammEndcap_TkEle_EtaBC_rate')

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib', 'HMvDRcylind5Calib', 'HMvDRcylind2p5Calib', 'HMvDRshapeCalib'], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [ ]:
hsets_new = []
labels_new = []
    
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEle'], 'EGq3EtaBC', None)
hsets_new.append(hsets[0])
labels_new.append("TkEle")

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEG'], 'EGq3EtaBCM1P2', None)
for hist in hsets:
    hist.normalize(31000)
hsets_new.append(hsets[0])
labels_new.append("p_{T}^{trk} > 2GeV")


hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEG'], 'EGq3EtaBCM1P5', None)
for hist in hsets:
    hist.normalize(31000)
hsets_new.append(hsets[0])
labels_new.append("p_{T}^{trk} > 5GeV")



hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEG'], 'EGq3EtaBCM1P10', None)
for hist in hsets:
    hist.normalize(31000)
hsets_new.append(hsets[0])
labels_new.append("p_{T}^{trk} > 10GeV")


draw([his.h_pt for his in hsets_new], labels_new, text=text, logy=True)


In [ ]:
for tp_sel in tp_select['DEF']:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V8', 'V9'], 'PU200', ['DEF','HMvDR'], tp_sel, None)
    for hist in hsets:
        hist.normalize(31000)
    draw([his.h_pt for his in hsets], labels, text=text, logy=True)
     

In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', tp, ['EmEtaABC', 'EmEtaBC'], None)
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    hist_list = [his.h_pt for his in hsets]
    draw(hist_list, labels, text=text, logy=True, do_ratio=True)
#     hRatio = hist_list[1].Clone(uuid.uuid4().hex[:6])
#     stuff.append(hRatio)
#     hRatio.Divide(hist_list[0])
#     drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', ['DEF','DEFCalib'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
drawSame(hist_list, labels, text=text, logy=True)
hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
stuff.append(hRatio)
hRatio.Divide(hist_list[1])
drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', ['DEF','DEFMerged'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
draw(hist_list, labels, text=text, logy=True, do_ratio = True)
# hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
# stuff.append(hRatio)
# hRatio.Divide(hist_list[1])
# drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

### Comparisono baseline vs Manos EGID

In [ ]:
for tp in ['DEF']:
    for etaregion in [['EmEtaBC', 'Emv1EtaBC']]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', tp, etaregion, None)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        hist_list = [his.h_pt for his in hsets]
        drawSame(hist_list, labels, text=text, logy=True)
        hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
        stuff.append(hRatio)
        hRatio.Divide(hist_list[1])
        drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        # drawSame([his.h_ptVabseta for his in hsets], labels, text=text)
  

In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 11) for his in hsets], labels, text=text+', p_{T}^{L1}>10GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')



In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 21) for his in hsets], labels, text=text+', p_{T}^{L1}>20GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 31) for his in hsets], labels, text=text+', p_{T}^{L1}>30GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 41) for his in hsets], labels, text=text+', p_{T}^{L1}>40GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')
